In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from astropy.cosmology import LambdaCDM

from scipy.optimize import curve_fit

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

In [9]:
# calling necessary class from ler package
from ler.gw_source_population import CBCSourceRedshiftDistribution

# uncomment the following line to see the docstring
# SourceGalaxyPopulationModel?

# class initialization
# default model "BBH popI/II Oguri2018"
cbc = CBCSourceRedshiftDistribution()

# list out the models for the merger rate density wrt redshift
print("\n available model list with it input parameters: \n", cbc.merger_rate_density_model_list)

z_to_luminosity_distance interpolator will be loaded from ./interpolator_pickle/z_to_luminosity_distance/z_to_luminosity_distance_3.pickle
differential_comoving_volume interpolator will be loaded from ./interpolator_pickle/differential_comoving_volume/differential_comoving_volume_1.pickle
merger_rate_density interpolator will be loaded from ./interpolator_pickle/merger_rate_density/merger_rate_density_2.pickle

 available model list with it input parameters: 
 {'merger_rate_density_bbh_popI_II_oguri2018': {'R0': 2.39e-08, 'b2': 1.6, 'b3': 2.0, 'b4': 30}, 'star_formation_rate_madau_dickinson2014': {'af': 2.7, 'bf': 5.6, 'cf': 2.9}, 'merger_rate_density_bbh_popIII_ken2022': {'n0': 1.92e-08, 'aIII': 0.66, 'bIII': 0.3, 'zIII': 11.6}, 'merger_rate_density_bbh_primordial_ken2022': {'n0': 4.4e-11, 't0': 13.786885302009708}}


In [5]:
# let's find mtot_max, let z_max = 2
# say ler default BNS mass model has a cut off 2.3 Mo
# mtot_max = 2.3*2*(1+2) = 13.8 

In [2]:
from ler.rates import LeR
ler = LeR(verbose=False)
# ler.available_gw_prior_list_and_its_params

In [10]:
ler = LeR(npool=8, event_type = 'BNS', mtot_max=14, z_max = 2.0,
    # for source parameters
    source_priors=dict(
        merger_rate_density='merger_rate_density_bbh_popI_II_oguri2018',
        source_frame_masses='binary_masses_BNS_bimodal',
    ),

    # None means default values
    source_priors_params = dict( 
       merger_rate_density = None, 
       source_frame_masses=None
    ),
       
    # for lens parameters
    lens_functions=dict(
        strong_lensing_condition="rjs_with_cross_section_SIS",
        optical_depth="optical_depth_SIS_haris",
    ),
    lens_priors=dict(
        velocity_dispersion="velocity_dispersion_gengamma",
    ),
    lens_priors_params=dict(
        velocity_dispersion=dict(a=2.32 / 2.67, c=2.67)
    ),
    
    # for snr generation
    waveform_approximant = 'IMRPhenomXPHM',
    snr_type='ann',
    # snr_type="inner_product",
    spin_zero=False,
    spin_precession=True,
    # change psd accordingly. I am providing default from bilby
    psds = dict( L1 = 'aLIGO_O4_high_asd.txt', H1 =  'aLIGO_O4_high_asd.txt'  ),
    # psds = dict( L1 = 'aligo_O4low_asd.txt', H1 =  'aligo_O4low_asd.txt'  ),
    # You don't need to provide ifos if you have given psds with detector name(s). Default detector orientation will be considered. For new detectors, you might want to pride detector location and orientation in ifos. 
    # ifos = ['L1'], 'H1'],
    verbose=False,
    )

In [28]:
ler = LeR(npool=6,
          verbose=False,
            z_max = 2.0,
            waveform_approximant = 'IMRPhenomXPHM',
            snr_type='ann',
            spin_zero=False,
            spin_precession=True,
            psds = dict( L1 = 'aLIGO_O4_high_asd.txt', H1 =  'aLIGO_O4_high_asd.txt'  ),)

z_to_luminosity_distance interpolator will be loaded from ./interpolator_pickle/z_to_luminosity_distance/z_to_luminosity_distance_4.pickle
differential_comoving_volume interpolator will be loaded from ./interpolator_pickle/differential_comoving_volume/differential_comoving_volume_6.pickle
merger_rate_density interpolator will be generated at ./interpolator_pickle/merger_rate_density/merger_rate_density_5.pickle
z_to_Dc interpolator will be loaded from ./interpolator_pickle/z_to_Dc/z_to_Dc_3.pickle
Dc_to_z interpolator will be loaded from ./interpolator_pickle/Dc_to_z/Dc_to_z_3.pickle
angular_diameter_distance interpolator will be loaded from ./interpolator_pickle/angular_diameter_distance/angular_diameter_distance_3.pickle
differential_comoving_volume interpolator will be loaded from ./interpolator_pickle/differential_comoving_volume/differential_comoving_volume_7.pickle
velocity_dispersion_ewoud interpolator will be generated at ./interpolator_pickle/velocity_dispersion_ewoud/velocity

interpolation for each mass_ratios:   2%|▌                           | 1/50 [00:03<03:11,  3.91s/it]


KeyboardInterrupt: 

In [18]:
ler.batch_size = 1000
unlensed_params = ler.unlensed_cbc_statistics(size=1000)

unlensed params will be store in ./ler_data/unlensed_param.json
chosen batch size = 1000 with total size = 1000
There will be 1 batche(s)
Batch no. 1
sampling gw source params...
calculating snrs...
saving all unlensed_params in ./ler_data/unlensed_param.json...


In [19]:
unlensed_params.keys()

dict_keys(['zs', 'geocent_time', 'ra', 'dec', 'phase', 'psi', 'theta_jn', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'luminosity_distance', 'mass_1_source', 'mass_2_source', 'mass_1', 'mass_2', 'L1', 'H1', 'optimal_snr_net'])

In [20]:
y_pred = unlensed_params['optimal_snr_net']

In [21]:
y_test = ler.snr_bilby(gw_param_dict=unlensed_params)['optimal_snr_net']

100%|███████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 74.74it/s]


In [23]:
#y_pred = (y_pred > 0.5)
idx = (y_test>1)
#idx = np.arange(10000)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)[idx][10:25])

[[1.06854673 1.24419614]
 [1.37490208 1.68265384]
 [0.95982928 1.14031388]
 [0.98142596 1.16366489]
 [1.05617352 1.26866148]
 [0.91448617 1.05234255]
 [1.32851566 1.58681619]
 [0.98979227 1.24024895]
 [1.00198222 1.19432353]
 [0.94533098 1.13739133]
 [1.67255392 2.05732739]
 [1.06001657 1.24702492]
 [1.0052715  1.26751638]
 [0.93049549 1.08662441]
 [0.93641905 1.1340932 ]]


In [27]:
y_pred[idx]+0.3-y_test[idx]

array([ 0.00074468,  0.12121519,  0.11198469,  0.11608075,  0.08844948,
        0.05752745,  0.09505743,  0.0828945 , -0.20748165,  0.13999832,
        0.12435059, -0.00775176,  0.1195154 ,  0.11776107,  0.08751204,
        0.16214362,  0.04169947,  0.04954332,  0.10765868,  0.10793965,
       -0.08477346,  0.11299165,  0.03775512,  0.14387108,  0.10232584,
        0.17094215,  0.07441481, -0.12908522,  0.15674684, -0.04429127,
        0.16296299])

In [13]:
ler.selecting_n_unlensed_detectable_events(
    size=500, 
    batch_size=100000, 
    snr_threshold=4.0, 
    output_jsonfile='n_unlensed_detectable_bns_neha.json', meta_data_file='n_unlensed_detectable_bns_meta_neha.json', 
    resume=False,
    snr_recalculation=True,
    snr_threshold_recalculation=2, # error margein of 2
    );

collected number of detectable events =  0


100%|█████████████████████████████████████████████████████████████| 542/542 [00:15<00:00, 34.51it/s]


collected number of detectable events =  56
total number of events =  100000
total unlensed rate (yr^-1): 18.883214018812208


100%|█████████████████████████████████████████████████████████████| 562/562 [00:16<00:00, 34.35it/s]


collected number of detectable events =  111
total number of events =  200000
total unlensed rate (yr^-1): 18.71461389364424
storing detectable unlensed params in ./ler_data/n_unlensed_detectable_bns_neha.json

 trmming final result to size=100


In [11]:
ler.selecting_n_unlensed_detectable_events(
    size=500, 
    batch_size=100000, 
    snr_threshold=4.0, 
    output_jsonfile='n_unlensed_detectable_bns_neha_ann.json', meta_data_file='n_unlensed_detectable_bns_meta_neha_ann.json', 
    resume=True,
    #snr_recalculation=True,
    #snr_threshold_recalculation=2.,
    );

collected number of detectable events =  0
collected number of detectable events =  30
total number of events =  100000
total unlensed rate (yr^-1): 10.116007510077969
collected number of detectable events =  63
total number of events =  200000
total unlensed rate (yr^-1): 10.621807885581868
collected number of detectable events =  98
total number of events =  300000
total unlensed rate (yr^-1): 11.015208177640455
collected number of detectable events =  140
total number of events =  400000
total unlensed rate (yr^-1): 11.80200876175763
collected number of detectable events =  163
total number of events =  500000
total unlensed rate (yr^-1): 10.992728160951392
collected number of detectable events =  192
total number of events =  600000
total unlensed rate (yr^-1): 10.790408010749832
collected number of detectable events =  223
total number of events =  700000
total unlensed rate (yr^-1): 10.742236546416128
collected number of detectable events =  258
total number of events =  800000
t

In [7]:
# only with inner product
ler = LeR(npool=8, event_type = 'BNS', mtot_max=14, z_max = 2.0,
    # for source parameters
    source_priors=dict(
        merger_rate_density='merger_rate_density_bbh_popI_II_oguri2018',
        source_frame_masses='binary_masses_BNS_bimodal',
    ),

    # None means default values
    source_priors_params = dict( 
       merger_rate_density = None, 
       source_frame_masses=None
    ),
       
    # for lens parameters
    lens_functions=dict(
        strong_lensing_condition="rjs_with_cross_section_SIS",
        optical_depth="optical_depth_SIS_haris",
    ),
    lens_priors=dict(
        velocity_dispersion="velocity_dispersion_gengamma",
    ),
    lens_priors_params=dict(
        velocity_dispersion=dict(a=2.32 / 2.67, c=2.67)
    ),
    
    # for snr generation
    waveform_approximant = 'IMRPhenomXPHM',
    snr_type="inner_product",
    spin_zero=False,
    spin_precession=True,
    psds = dict( L1 = 'aLIGO_O4_high_asd.txt', H1 =  'aLIGO_O4_high_asd.txt'  ),
    verbose=False,
    )

In [8]:
ler.selecting_n_unlensed_detectable_events(
    size=50, 
    batch_size=10000, 
    snr_threshold=4.0, 
    output_jsonfile='n_unlensed_detectable_bns_neha.json', meta_data_file='n_unlensed_detectable_bns_meta_neha.json', 
    resume=True,
    );

collected number of detectable events =  0


  0%|                                                                     | 0/10000 [00:00<?, ?it/s]

collected number of detectable events =  5
total number of events =  10000
total unlensed rate (yr^-1): 16.860012516796616


  0%|                                                                     | 0/10000 [00:00<?, ?it/s]

collected number of detectable events =  10
total number of events =  20000
total unlensed rate (yr^-1): 16.860012516796616


...

collected number of detectable events =  19
total number of events =  40000
total unlensed rate (yr^-1): 16.017011890956784


  0%|                                                                     | 0/10000 [00:00<?, ?it/s]

collected number of detectable events =  21
total number of events =  50000
total unlensed rate (yr^-1): 14.162410514109155


  1%|▌                                                           | 91/10000 [00:03<06:41, 24.68it/s]


KeyboardInterrupt: 

In [24]:
ler.selecting_n_lensed_detectable_events(size=5, batch_size=1000, snr_threshold=8.0, num_img=2, output_jsonfile='n_lensed_detectable_bns_neha.json', meta_data_file='n_lensed_detectable_bns_meta_neha.json', resume=True);

collected number of detectable events =  0


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of events =  1
total number of events =  1000
total lensed rate (yr^-1): 0.01091708098518182


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of events =  1
total number of events =  2000
total lensed rate (yr^-1): 0.00545854049259091


...

collected number of events =  2
total number of events =  15000
total lensed rate (yr^-1): 0.0014556107980242428


  0%|                                                                      | 0/1000 [00:00<?, ?it/s]

collected number of events =  2
total number of events =  16000
total lensed rate (yr^-1): 0.0013646351231477276


  5%|██▉                                                          | 49/1000 [00:03<01:00, 15.59it/s]


KeyboardInterrupt: 